# **Web Usage**

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import display
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
nama_file = '/content/drive/MyDrive/PPW/webuage.csv'
df = pd.read_csv(nama_file, sep=None, engine="python")

In [14]:
df

,Remote host,Remote logname,Remote user,Request time,Request method,Request URI,Request Protocol,Status,Size of response (incl. headers)
0,65.55.147.227,-,-,2009-10-15T02:00:24Z,GET,/index.html,HTTP/1.1,200,21878
1,65.55.86.34,-,-,2009-10-15T02:00:58Z,GET,/index.html,HTTP/1.1,200,1416
2,148.188.55.88,-,-,2009-10-15T02:01:41Z,GET,/faq.html,HTTP/1.1,200,10946
3,72.30.57.238,-,-,2009-10-15T02:01:59Z,GET,/contribute.txt,HTTP/1.0,200,39943
4,66.249.139.233,-,-,2009-10-15T02:02:09Z,GET,/faq.html,HTTP/1.1,200,17247
...,...,...,...,...,...,...,...,...,...
132253,66.249.196.230,-,-,2009-10-22T01:58:21Z,GET,/contribute.txt,HTTP/1.1,200,16569
132254,66.249.41.85,-,-,2009-10-22T01:58:22Z,GET,/contribute.txt,HTTP/1.1,200,380
132255,65.55.236.245,-,-,2009-10-22T01:59:33Z,GET,/index.html,HTTP/1.1,200,6695
132256,65.55.198.69,-,-,2009-10-22T01:59:34Z,GET,/contribute.txt,HTTP/1.1,200,47672


## **1. Normalisasi nama kolom agar mudah diakses**


In [15]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
print("Kolom yang tersedia:", df.columns.tolist())



Kolom yang tersedia: ['remote_host', 'remote_logname', 'remote_user', 'request_time', 'request_method', 'request_uri', 'request_protocol', 'status', 'size_of_response_(incl._headers)']


## **2. Filter**

In [16]:
filtered = df[
    (df["request_method"] == "GET") &
    (df["request_uri"].isin(["/faq.html", "/index.html"])) &
    (df["status"] == 200)
]

In [18]:
print(f"Jumlah data hasil filter: {len(filtered)}")
display(filtered.head())

Jumlah data hasil filter: 75656


,remote_host,remote_logname,remote_user,request_time,request_method,request_uri,request_protocol,status,size_of_response_(incl._headers)
0,65.55.147.227,-,-,2009-10-15T02:00:24Z,GET,/index.html,HTTP/1.1,200,21878
1,65.55.86.34,-,-,2009-10-15T02:00:58Z,GET,/index.html,HTTP/1.1,200,1416
2,148.188.55.88,-,-,2009-10-15T02:01:41Z,GET,/faq.html,HTTP/1.1,200,10946
4,66.249.139.233,-,-,2009-10-15T02:02:09Z,GET,/faq.html,HTTP/1.1,200,17247
5,72.30.50.248,-,-,2009-10-15T02:02:13Z,GET,/index.html,HTTP/1.0,200,7883


In [31]:
output_file = "hasil_filtered_webuage.csv"
filtered.to_csv(output_file, index=False)

## **3. Cek Duplikasi**

In [23]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

#Tampilkan jumlah duplikasi berdasarkan IP
dupe_count = df["remote_host"].value_counts()
print("Jumlah kemunculan setiap IP:")
display(dupe_count.head(10))  # tampilkan 10 IP teratas

#Tampilkan hanya IP yang muncul lebih dari 1 kali
dupe_ips = dupe_count[dupe_count > 1]
print(f"Jumlah IP yang duplikat: {len(dupe_ips)}")
display(dupe_ips)

Jumlah kemunculan setiap IP:


,count
remote_host,
66.249.63.228,4
66.249.213.145,3
66.249.242.12,3
66.249.149.62,3
66.249.127.53,3
134.34.81.143,3
65.55.203.175,3
66.249.102.3,3
65.55.22.206,3


Jumlah IP yang duplikat: 2306


,count
remote_host,
66.249.63.228,4
66.249.213.145,3
66.249.242.12,3
66.249.149.62,3
66.249.127.53,3
...,...
66.249.129.145,2
212.126.198.72,2
66.249.159.29,2


In [28]:
# === Konversi kolom waktu menjadi datetime agar bisa diurutkan ===
df["request_time"] = pd.to_datetime(df["request_time"], errors="coerce")

# === Urutkan berdasarkan IP dan waktu ===
df_sorted = df.sort_values(by=["remote_host", "request_time"]).reset_index(drop=True)

# === Filter sesuai kriteria ===
filtered = df_sorted[
    (df_sorted["request_method"] == "GET") &
    (df_sorted["request_uri"].isin(["/faq.html", "/index.html"])) &
    (df_sorted["status"] == 200)
]

# === Urutkan hasil akhir berdasarkan IP (Remote Host) ===
filtered = filtered.sort_values(by="remote_host").reset_index(drop=True)

In [29]:
# === Tampilkan hasil ===
print(f"Jumlah data hasil filter: {len(filtered)}")
display(filtered.head(20))  # tampilkan 20 baris teratas

# === 9️⃣ Simpan hasil ke file baru dan download ===
output_file = "filtered_log_sorted_by_ip.csv"
filtered.to_csv(output_file, index=False)

Jumlah data hasil filter: 75656


,remote_host,remote_logname,remote_user,request_time,request_method,request_uri,request_protocol,status,size_of_response_(incl._headers)
0,109.192.104.209,-,-,2009-10-18 18:27:27+00:00,GET,/index.html,HTTP/1.1,200,19395
1,109.192.104.86,-,-,2009-10-18 18:27:27+00:00,GET,/index.html,HTTP/1.1,200,99209
2,109.192.109.233,-,-,2009-10-18 18:26:27+00:00,GET,/faq.html,HTTP/1.1,200,757
3,109.192.111.243,-,-,2009-10-18 20:59:37+00:00,GET,/index.html,HTTP/1.1,200,10937
4,109.192.117.151,-,-,2009-10-18 18:26:27+00:00,GET,/index.html,HTTP/1.1,200,285
5,109.192.121.165,-,-,2009-10-18 18:26:27+00:00,GET,/faq.html,HTTP/1.1,200,205
6,109.192.135.2,-,-,2009-10-18 20:58:33+00:00,GET,/index.html,HTTP/1.1,200,1127
7,109.192.138.239,-,-,2009-10-18 20:58:33+00:00,GET,/faq.html,HTTP/1.1,200,20926
8,109.192.143.138,-,-,2009-10-18 18:26:27+00:00,GET,/faq.html,HTTP/1.1,200,304620
9,109.192.147.1,-,-,2009-10-18 20:58:44+00:00,GET,/faq.html,HTTP/1.1,200,17723


## **4. Data lengkap yang mengandung duplikasi**

In [30]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

dupe_count = df["remote_host"].value_counts()
print("📊 Jumlah kemunculan setiap IP:")
display(dupe_count.head(10))  # tampilkan 10 IP teratas

dupe_ips = dupe_count[dupe_count > 1]
print(f"\n🔍 Jumlah IP yang duplikat: {len(dupe_ips)}")
display(dupe_ips)

duplicates = df[df["remote_host"].isin(dupe_ips.index)]
print("\n📋 Data lengkap yang mengandung duplikasi:")
display(duplicates.head(20))  # tampilkan 20 baris pertama

📊 Jumlah kemunculan setiap IP:


,count
remote_host,
66.249.63.228,4
66.249.213.145,3
66.249.242.12,3
66.249.149.62,3
66.249.127.53,3
134.34.81.143,3
65.55.203.175,3
66.249.102.3,3
65.55.22.206,3



🔍 Jumlah IP yang duplikat: 2306


,count
remote_host,
66.249.63.228,4
66.249.213.145,3
66.249.242.12,3
66.249.149.62,3
66.249.127.53,3
...,...
66.249.129.145,2
212.126.198.72,2
66.249.159.29,2



📋 Data lengkap yang mengandung duplikasi:


,remote_host,remote_logname,remote_user,request_time,request_method,request_uri,request_protocol,status,size_of_response_(incl._headers)
7,66.249.61.232,-,-,2009-10-15 02:02:39+00:00,GET,/contribute.txt,HTTP/1.1,200,10946
8,65.55.80.97,-,-,2009-10-15 02:02:51+00:00,GET,/index.html,HTTP/1.1,200,1416
29,65.55.247.6,-,-,2009-10-15 02:03:41+00:00,GET,/index.html,HTTP/1.1,200,1202
93,65.55.6.89,-,-,2009-10-15 02:06:54+00:00,GET,/somefile.zip,HTTP/1.1,200,26936
127,65.55.143.49,-,-,2009-10-15 02:12:58+00:00,GET,/contribute.txt,HTTP/1.1,200,269198
155,66.249.110.240,-,-,2009-10-15 02:23:34+00:00,GET,/somefile.zip,HTTP/1.1,200,1416
167,65.55.218.159,-,-,2009-10-15 02:30:51+00:00,GET,/index.html,HTTP/1.1,200,16568
283,65.55.89.151,-,-,2009-10-15 02:36:59+00:00,GET,/robots.txt,HTTP/1.1,200,1590
291,65.55.25.110,-,-,2009-10-15 02:39:01+00:00,GET,/contribute.txt,HTTP/1.1,200,788
292,65.55.145.135,-,-,2009-10-15 02:39:07+00:00,GET,/faq.html,HTTP/1.1,200,205


In [35]:
nama_file2 = '/content/drive/MyDrive/PPW/webuageOutput /filtered_log_sorted_by_ip.csv'
df2 = pd.read_csv(nama_file2, sep=None, engine="python")

In [36]:
# === Normalisasi nama kolom ===
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# === Konversi waktu ke datetime ===
df["request_time"] = pd.to_datetime(df["request_time"], errors="coerce")

# === Filter hanya method GET dan halaman index/faq ===
filtered = df[
    (df["request_method"] == "GET") &
    (df["request_uri"].isin(["/faq.html", "/index.html"])) &
    (df["status"] == 200)
].copy()

# === Ganti halaman menjadi A/B ===
filtered["url"] = filtered["request_uri"].replace({
    "/index.html": "A",
    "/faq.html": "B"
})

# === Urutkan berdasarkan IP dan waktu ===
filtered = filtered.sort_values(by=["remote_host", "request_time"]).reset_index(drop=True)

# === Hitung selisih waktu antarakses (per IP) ===
filtered["selisih_menit"] = (
    filtered.groupby("remote_host")["request_time"]
    .diff()
    .dt.total_seconds()
    .div(60)
    .fillna(0)
    .round(2)
)

# === Buat kolom sesi per IP (gap ≥ 30 menit = sesi baru)
session_timeout = 30  # menit
filtered["sesi"] = (
    filtered.groupby("remote_host")["selisih_menit"]
    .transform(lambda x: (x > session_timeout).cumsum() + 1)
)

# === Pilih kolom utama dan tampilkan ===
final_table = filtered[[
    "remote_host", "request_time", "url", "status",
    "size_of_response_(incl._headers)", "selisih_menit", "sesi"
]].rename(columns={"remote_host": "ip"})

print("Semua data ditampilkan dengan pembagian sesi per 30 menit:")
display(final_table.head(20))

# === Simpan hasil ke file CSV ===
output_file = "sessionized_full_log_fixed.csv"
final_table.to_csv(output_file, index=False)

Semua data ditampilkan dengan pembagian sesi per 30 menit:


,ip,request_time,url,status,size_of_response_(incl._headers),selisih_menit,sesi
0,109.192.104.209,2009-10-18 18:27:27+00:00,A,200,19395,0.0,1
1,109.192.104.86,2009-10-18 18:27:27+00:00,A,200,99209,0.0,1
2,109.192.109.233,2009-10-18 18:26:27+00:00,B,200,757,0.0,1
3,109.192.111.243,2009-10-18 20:59:37+00:00,A,200,10937,0.0,1
4,109.192.117.151,2009-10-18 18:26:27+00:00,A,200,285,0.0,1
5,109.192.121.165,2009-10-18 18:26:27+00:00,B,200,205,0.0,1
6,109.192.135.2,2009-10-18 20:58:33+00:00,A,200,1127,0.0,1
7,109.192.138.239,2009-10-18 20:58:33+00:00,B,200,20926,0.0,1
8,109.192.143.138,2009-10-18 18:26:27+00:00,B,200,304620,0.0,1
9,109.192.147.1,2009-10-18 20:58:44+00:00,B,200,17723,0.0,1


In [37]:
# Pastikan final_table sudah berisi ip, request_time, url, selisih_menit
df2 = final_table.copy()

# === Tentukan batas sesi (30 menit) ===
session_timeout = pd.Timedelta(minutes=30)

# === Urutkan berdasarkan IP dan waktu ===
df2 = df2.sort_values(by=["ip", "request_time"]).reset_index(drop=True)

# === Buat kolom sesi berdasarkan gap waktu antar akses per IP ===
df2["sesi"] = (
    df2.groupby("ip")["request_time"]
    .diff()
    .gt(session_timeout)
    .groupby(df2["ip"])
    .cumsum()
    .add(1)
)

# === Tandai halaman A dan B dalam setiap sesi ===
df2["A"] = (df2["url"] == "A").astype(int)
df2["B"] = (df2["url"] == "B").astype(int)

# === Rekap per IP dan sesi (gabungkan kunjungan dalam satu periode)
result = (
    df2.groupby(["ip", "sesi"], as_index=False)[["A", "B"]]
    .max()  # jika pernah ke A atau B dalam sesi tersebut = 1
)

# === Hapus kolom sesi agar tampil seperti log ringkas ===
result = result.drop(columns=["sesi"])

# === Tampilkan hasil akhir ===
print("Ringkasan kunjungan per IP (setiap gap ≥ 30 menit buat baris baru):")
display(result.head(20))

# === Simpan ke CSV ===
result.to_csv("ip_page_session_30min.csv", index=False)

Ringkasan kunjungan per IP (setiap gap ≥ 30 menit buat baris baru):


,ip,A,B
0,109.192.104.209,1,0
1,109.192.104.86,1,0
2,109.192.109.233,0,1
3,109.192.111.243,1,0
4,109.192.117.151,1,0
5,109.192.121.165,0,1
6,109.192.135.2,1,0
7,109.192.138.239,0,1
8,109.192.143.138,0,1
9,109.192.147.1,0,1


## **5. Mencari IP yang sering muncul lebih dari sekali**

In [38]:
# === Periksa IP yang muncul lebih dari satu kali ===
ip_counts = result["ip"].value_counts()

# Ambil hanya IP yang duplikat (muncul > 1 kali)
dupe_ips = ip_counts[ip_counts > 1]
print(f"Jumlah IP duplikat: {len(dupe_ips)}")
display(dupe_ips)

# === Tampilkan baris lengkap dari IP yang duplikat ===
dupe_rows = result[result["ip"].isin(dupe_ips.index)]
print("Data lengkap untuk IP yang duplikat:")
display(dupe_rows)


Jumlah IP duplikat: 734


,count
ip,
65.55.157.143,3
66.249.99.24,3
66.249.60.199,3
65.55.251.19,3
65.55.163.84,3
...,...
66.249.33.148,2
65.55.2.41,2
66.249.206.235,2


Data lengkap untuk IP yang duplikat:


,ip,A,B
3323,129.132.208.145,1,0
3324,129.132.208.145,0,1
5996,134.214.87.164,1,0
5997,134.214.87.164,1,0
6136,134.226.139.239,1,0
...,...,...,...
72428,91.23.240.235,1,0
72522,91.23.26.33,1,0
72523,91.23.26.33,1,0
72683,91.23.52.142,1,0
